In [27]:
import sqlite3
import pandas as pd
from transformers import BertTokenizer, BertForQuestionAnswering
import transformers
import torch


transformers.logging.set_verbosity_error()

In [17]:
query = "How long do Hamsters live?"

selected_ids = str(("Animal testing on Syrian hamsters", "Domestication of the Syrian hamster",
                "Golden hamster", "Hamster", "The Hamsters (album)"))

db_path = "data/wikipedia/docs.db"
connection = sqlite3.connect(db_path, check_same_thread=False)
cursor = connection.cursor()

cursor.execute("SELECT id, text FROM documents WHERE id IN " + selected_ids)
data_json = {"id": [], "text": []}
for r in cursor.fetchall():
    data_json["id"].append(r[0]); data_json["text"].append(r[1])

cursor.close()
connection.close()

examples = [[data_json['text'][idx], query] for idx in range(len(data_json["text"]))]

print("Query: ", query)
print("Relevant documents:")
display(pd.DataFrame.from_dict(data_json).head(5))

Query:  How long do Hamsters live?
Relevant documents:


,id,text
0,Animal testing on Syrian hamsters,Animal testing on Syrian hamsters\n\nSyrian ha...
1,Domestication of the Syrian hamster,Domestication of the Syrian hamster\n\nThe dom...
2,Golden hamster,"Golden hamster\n\nThe golden hamster, or Syria..."
3,Hamster,Hamster\n\nHamsters are rodents belonging to t...
4,The Hamsters (album),The Hamsters (album)\n\nThe Hamsters (1993) (k...


In [33]:
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

final_answer = ""
max_start_score = max_end_score = 0

for example in examples:
    question, text = example[1], example[0]
    inputs = tokenizer(question, text, return_tensors="pt", max_length=512, truncation=True)

    outputs = model(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    input_ids = inputs["input_ids"].tolist()[0]
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) +1

    answer = tokenizer.decode(input_ids[answer_start:answer_end])
    
    if torch.max(start_scores) > max_start_score and \
        torch.max(end_scores) > max_end_score:
        final_answer = answer
        max_start_score = torch.max(start_scores)
        max_end_score = torch.max(end_scores)
    
print(final_answer)

two to three years
